In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.2311, 0.4787, 0.5608],
        [0.2162, 0.4026, 0.2341],
        [0.3720, 0.0705, 0.7584],
        [0.7157, 0.6610, 0.3376],
        [0.0018, 0.3098, 0.1306]])


In [2]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
classifier('We are very angry to show you the 🤗 Transformers library.')

[{'label': 'NEGATIVE', 'score': 0.9994889497756958}]

In [5]:
meter = ['4/4', '6/8', '2/4', '3/4', '9/8', '12/8', '3/2']
mode = ['Gmajor',
 'Dmajor',
 'Amajor',
 'Adorian',
 'Eminor',
 'Edorian',
 'Bminor',
 'Amixolydian',
 'Aminor',
 'Dmixolydian',
 'Cmajor',
 'Fmajor',
 'Dminor',
 'Gminor',
 'Ddorian',
 'Gdorian',
 'Emajor',
 'Gmixolydian',
 'Bdorian',
 'Cdorian',
 'Fdorian',
 'Emixolydian',
 'Bmixolydian']

In [131]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())
tokenizer.add_special_tokens(meter + mode)

30

In [132]:
from tokenizers.pre_tokenizers import ByteLevel

tokenizer.pre_tokenizer = ByteLevel()

In [133]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(min_frequency=2, special_tokens=meter + mode)
tokenizer.train(files=["cleaned_tunes.txt"], trainer=trainer)

In [134]:
output = tokenizer.encode("2/4 Gmajor |:d>gfe|dBAG|E/F/GE/F/G|BAGF|d>gfe|dBAG|E/F/GE/F/G|BAG2:||:e>fga|bagf|e/f/ge/f/g|d/c/B/A/BA|e>fga|bagf|e/f/ge/f/g|agg2:|")
print(output.tokens)

['2/4', 'Ġ', 'Gmajor', 'Ġ|:', 'd', '>', 'gfe', '|', 'dBAG', '|', 'E', '/', 'F', '/', 'GE', '/', 'F', '/', 'G', '|', 'BAGF', '|', 'd', '>', 'gfe', '|', 'dBAG', '|', 'E', '/', 'F', '/', 'GE', '/', 'F', '/', 'G', '|', 'BAG', '2', ':||:', 'e', '>', 'fga', '|', 'bagf', '|', 'e', '/', 'f', '/', 'ge', '/', 'f', '/', 'g', '|', 'd', '/', 'c', '/', 'B', '/', 'A', '/', 'BA', '|', 'e', '>', 'fga', '|', 'bagf', '|', 'e', '/', 'f', '/', 'ge', '/', 'f', '/', 'g', '|', 'agg', '2', ':|']


In [10]:
tokenizer.save("tokenizer-tune.json")

In [11]:
tokenizer.alphabet()

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'alphabet'

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./models/EsperBERTo-small/vocab.json",
    "./models/EsperBERTo-small/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(
    tokenizer.encode("Mi estas Julien.")
)
# Encoding(num_tokens=7, ...)
# tokens: ['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

In [ ]:
from torch.utils.data import Dataset

class TunesDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "./models/EsperBERTo-small/vocab.json",
            "./models/EsperBERTo-small/merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []

        src_files = Path("./data/").glob("*-eval.txt") if evaluate else Path("./data/").glob("*-train.txt")
        for src_file in src_files:
            print("🔥", src_file)
            lines = src_file.read_text(encoding="utf-8").splitlines()
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

In [12]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

In [15]:
model = BertModel.from_pretrained("bert-base-cased")


KeyboardInterrupt: 

In [22]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer("Hello world")['input_ids']
tokenizer(" Hello world")['input_ids']


[18435, 995]

In [17]:
tokenizer

PreTrainedTokenizer(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [18]:
from tokenizers.pre_tokenizers import ByteLevel

TypeError: Can't convert 's' to PyBool

In [135]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()

In [136]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(files=["cleaned_tunes.txt"], trainer=trainer)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 25000


In [137]:
tokenizer.model.save('.')

['./vocab.json', './merges.txt']

In [138]:
test_tune = 'M:6/8 K:Eminor e|:edeB2A|AGBAGG|edeB2A|ABde2e|edeB2A|AGBAGB|GABd2G|1ABde2e:|2ABde2A|]|:edeg2a-|aedBAG|edeg2a-|aede3|edeg2a-|aedBAB|GABd2G|1ABde2e:|2ABde2|]'

In [152]:
encoding = tokenizer(test_tune)

In [153]:
print("Encoded string: {}".format(encoding.tokens))

Encoded string: <bound method BatchEncoding.tokens of {'input_ids': [21, 14, 23, 37912, 273, 304, 91, 25, 18654, 33, 17, 32, 91, 4760, 4339, 11190, 91, 18654, 33, 17, 32, 91, 6242, 2934, 17, 68, 91, 18654, 33, 17, 32, 91, 4760, 4339, 4579, 91, 38, 6242, 67, 17, 38, 91, 16, 6242, 2934, 17, 68, 25, 91, 17, 6242, 2934, 17, 32, 91, 60, 91, 25, 276, 1533, 17, 64, 22831, 8432, 4339, 38, 91, 276, 1533, 17, 64, 22831, 8432, 68, 18, 91, 276, 1533, 17, 64, 22831, 8432, 4339, 33, 91, 38, 6242, 67, 17, 38, 91, 16, 6242, 2934, 17, 68, 25, 91, 17, 6242, 2934, 17, 91, 60], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>


In [142]:
decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Decoded string:  6/8 Eminor e|:edeB2A|AGBAGG|edeB2A|ABde2e|edeB2A|AGBAGB|GABd2G|1ABde2e:|2ABde2A|]|:edeg2a-|aedBAG|edeg2a-|aede3|edeg2a-|aedBAB|GABd2G|1ABde2e:|2ABde2|]


In [60]:
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-gpt2")

model = AutoModelForCausalLM.from_pretrained("sshleifer/tiny-gpt2")

In [143]:
encoding = tokenizer.encode(test_tune)
print("Encoded string: {}".format(encoding.tokens))

Encoded string: ['Ġ6', '/', '8', 'ĠEminor', 'Ġe', '|:', 'edeB', '2', 'A', '|', 'AG', 'BAGG', '|', 'edeB', '2', 'A', '|', 'ABde', '2', 'e', '|', 'edeB', '2', 'A', '|', 'AGBAGB', '|', 'GABd', '2', 'G', '|', '1', 'ABde', '2', 'e', ':|', '2', 'ABde', '2', 'A', '|]|:', 'edeg', '2', 'a', '-|', 'a', 'edBAG', '|', 'edeg', '2', 'a', '-|', 'aede', '3', '|', 'edeg', '2', 'a', '-|', 'a', 'edBAB', '|', 'GABd', '2', 'G', '|', '1', 'ABde', '2', 'e', ':|', '2', 'ABde', '2', '|]']


In [146]:
tokenizer.encode(test_tune).ids

[319,
 14,
 23,
 463,
 684,
 272,
 4148,
 17,
 32,
 91,
 317,
 1373,
 91,
 4148,
 17,
 32,
 91,
 817,
 17,
 68,
 91,
 4148,
 17,
 32,
 91,
 19546,
 91,
 460,
 17,
 38,
 91,
 16,
 817,
 17,
 68,
 265,
 17,
 817,
 17,
 32,
 1077,
 898,
 17,
 64,
 382,
 64,
 1175,
 91,
 898,
 17,
 64,
 382,
 15832,
 18,
 91,
 898,
 17,
 64,
 382,
 64,
 1253,
 91,
 460,
 17,
 38,
 91,
 16,
 817,
 17,
 68,
 265,
 17,
 817,
 17,
 364]

In [154]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="cleaned_tunes.txt",
    block_size=512,
)

In [159]:
dataset[3]

{'input_ids': tensor([   19,    14,    19,   347,  1084,   273,   304,  3838,  3856,  6242,
            33,    91,    68,  3838,    33,  2004,    33,    91,    68,  3838,
          3856,  6242,    33,    91,    38,    17,  6242,  2004,    33,    25,
            91,    68,    18,  4299,    70,    17,    91,    68,    18,    69,
          2004,    33,    91,    68,    18,  4299,    70,    17,    91,    38,
            17,  6242,  2004,    33,    91,    68,    18,  4299,    70,    17,
            91,    68,    18,    69,  2004,    33,    91,   891, 10502,    70,
            17,    91,    38,  6242,    38,  2004,    33, 15886])}

In [124]:
from datasets import load_dataset, GenerateMode

dataset = load_dataset('text', data_files='cleaned_tunes.txt', download_mode = GenerateMode.FORCE_REDOWNLOAD)


Using custom data configuration default-93471251da4760db


Dataset text downloaded and prepared to /Users/jckelly/.cache/huggingface/datasets/text/default-93471251da4760db/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [147]:
train = dataset['train']

In [149]:
encoded_dataset = dataset['train'].map(lambda x: tokenizer.encode(x['text']), batched=True)

TypeError: TextInputSequence must be str

In [63]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

NameError: name 'train_dataset' is not defined

In [151]:
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-gpt2")

model = AutoModelForCausalLM.from_pretrained("sshleifer/tiny-gpt2")

In [118]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 2)
    (wpe): Embedding(1024, 2)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (l

In [119]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
# Let's load a pretrained Bert model and a simple optimizer
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('distilbert-base-cased', return_dict=True)


In [ ]:
model

In [122]:
# Now let's train our model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.train().to(device)
for i, batch in enumerate(dataloader):
    batch.to(device)
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    model.zero_grad()
    print(f'Step {i} - loss: {loss:.3}')
    if i > 5:
        break

NameError: name 'dataloader' is not defined